In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load the data

data/

│-- 2018/

│   │-- image_1.npy

│   │-- image_2.npy

│   └-- ...

│-- 2024/

│   │-- image_1.npy

│   │-- image_2.npy

│   └-- ...

│-- masks/  (zone de construction)

│   │-- mask_1.npy

│   │-- mask_2.npy

│   └-- ...

Importe data from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# à voir si c'est bien fait en forme de liste

Get the data

In [ ]:
# data : 64 x 64 x 10bandes

data_dir_2018 = "/..."
data_dir_2024 = "/..."
mask_dir = "/..."


Prepare pytorch dataset

In [2]:
class DatasetCadastre(torch.utils.data.Dataset):
    def __init__(self, data_dir_2018, data_dir_2024, mask_dir, transform=None):
        self.data_2018 = data_dir_2018
        self.data_2024 = data_dir_2024
        self.mask_path = mask_dir
        self.transform = transform

    def __getitem__(self, index):
        # Chargement des données
        img_2018 = np.load(self.data_2018[index]).astype(np.float32) # (64, 64, 10)
        img_2024 = np.load(self.data_2024[index]).astype(np.float32)
        mask = np.load(self.mask_path[index]).astype(np.float32)

        # Ajustement de l'ordre pytorch (channels, size)
        img_2018 = np.transpose(img_2018, (2,0,1))
        img_2024 = np.transpose(img_2024, (2,0,1))
        mask = np.expand_dims(mask, axis=0)

        # Transformation en tensor pytorch
        img_2018 = torch.tensor(img_2018)
        img_2024 = torch.tensor(img_2024)
        mask = torch.tensor(mask)

        # Si transformation appliquée
        if self.transform:
          img_2018 = self.transform(img_2018)
          img_2024 = self.transform(img_2024)

        return img_2018, img_2024, mask

    def __len__(self):
        return len(self.data_2018)

# Dataset gobal --> à voir si on fait pas plutôt un dataset pour chaque donnée (2018, 204 et mask)
cadastre_dataset = DatasetCadastre(data_dir_2018, data_dir_2024, mask_dir)


NameError: name 'torch' is not defined

Prepare dataloader

In [ ]:
batch_size = 512
num_threads = 4

train_loader = torch.utils.data.DataLoader(
    dataset=cadastre_dataset, batch_size=batch_size, shuffle=True, num_workers=num_threads
)
valid_loader = torch.utils.data.DataLoader(
    dataset=cadastre_dataset, batch_size=batch_size, shuffle=False, num_workers=num_threads
)

Normalize data

In [ ]:
# à voir

# Compute NDVI

In [ ]:
def NDVI_computed(img_tensor):
    IR = img_tensor[...,:,:] # bande IR
    RED = img_tensor[...,:,:] # bande RED
    NDVI = (IR-RED)/(IR+RED)
    return NDVI

# for img_2018, img_2024, mask in train_loader:
#     NDVI_2018 = NDVI_computed(img_2018)
#     NDVI_2024 = NDVI_computed(img_2024)
#     break

# Model definition

In [ ]:
class Conv2DRegressionModel(nn.Module): # prédiction d'indices (NDVI, ...)
    def __init__(self, int_channels: int = 10):
        super(Conv2DRegressionModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(int_channels, 32, kernel_size=3, padding=1),
            nn.LazyBatchNorm2d(),
            # nn.BatchNorm2d(int_channels*2)
            nn.ReLU(),
            nn.MaxPool2d(2, kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.LazyBatchNorm2d(),
            # nn.BatchNorm2d(int_channels*4)
            nn.ReLU(),
            nn.MaxPool2d(2, kernel_size=2),
            nn.Flatten(),
            nn.Linear(128*8*8, 512),
            nn.ReLU(),
            nn.Linear(512, 64*64)
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        y = self.layers(X)
        y = y.view(-1, 64, 64) # sortie en image NDVI
        return y


# Train the model

In [ ]:
n_epochs = 10
learning_rate = 0.001

model = Conv2DRegressionModel()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_set_len = len(train_loader)
val_set_len = len(valid_loader)

train_loss_comp, val_loss_comp = [], []
train_loss_pred, val_loss_pred = [], []
train_loss_comp_pred, val_loss_comp_pred = [], []

for epoch in range(n_epochs):
    model.train()
    accu_comp = 0.0
    accu_pred = 0.0
    accu_comp_pred = 0.0

    for img_2018, img_2024, mask in train_loader:
        # img_2018, img_2024, mask = img_2018.to(device), img_2024.to(device), mask.to(device)
        # Forward pass
        img_2024_pred = model(img_2018) # image 2024 prédite à partir de 2018

        NDVI_2018 = NDVI_computed(img_2018) # NDVI 2018 calculé (vrai)
        NDVI_2024 = NDVI_computed(img_2024) # NDVI 2024 calculé (vrai)

        NDVI_2024_comp = NDVI_computed(img_2024_pred) # NDVI 2024 caclulé à partir des bandes prédites

        NDVI_2024_pred = ... # NDVI 2024 prédit par le réseau de neurones

        loss_comp = loss_fn(NDVI_2024_comp, NDVI_2024) # comparaison des NDVI avec calcul
        loss_pred = loss_fn(NDVI_2024_pred, NDVI_2024) # comparaison des NDVI avec prédiction
        loss_comp_pred = loss_fn(NDVI_2024_comp, NDVI_2024_pred) # comparaison des NDVI calculé et prédit

        accu_comp += loss_comp.item()
        accu_pred += loss_pred.item()
        accu_comp_pred += loss_comp_pred.item()

        # backward pass
        optimizer.zero_grad()
        loss_comp.backward()
        loss_pred.backward()
        loss_comp_pred.backward()
        optimizer.step()
    train_loss_comp.append(accu_comp / train_set_len)
    train_loss_pred.append(accu_pred / train_set_len)
    train_loss_comp_pred.append(accu_comp_pred / train_set_len)

    # Validation - no gradient & eval mode
    model.eval()
    accu_comp = 0.0
    accu_pred = 0.0
    accu_comp_pred = 0.0
    with torch.no_grad():
        for img_2018, img_2024, mask in valid_loader:
            # img_2018, img_2024, mask = img_2018.to(device), img_2024.to(device), mask.to(device)
            # Forward pass
            img_2024_pred = model(img_2018) # image 2024 prédite à partir de 2018
            NDVI_2018 = NDVI_computed(img_2018) # NDVI 2018 calculé (vrai)
            NDVI_2024 = NDVI_computed(img_2024) # NDVI 2024 calculé (vrai)
            NDVI_2024_comp = NDVI_computed(img_2024_pred) # NDVI 2024 caclulé à partir des bandes prédites
            NDVI_2024_pred = ... # NDVI 2024 prédit par le réseau de neurones

            loss_comp = loss_fn(NDVI_2024_comp, NDVI_2024) # comparaison des NDVI avec calcul
            loss_pred = loss_fn(NDVI_2024_pred, NDVI_2024) # comparaison des NDVI avec prédiction
            loss_comp_pred = loss_fn(NDVI_2024_comp, NDVI_2024_pred) # comparaison des NDVI calculé et prédit

            accu_comp += loss_comp.item()
            accu_pred += loss_pred.item()
            accu_comp_pred += loss_comp_pred.item()
    valid_loss_comp.append(accu_comp / valid_set_len)
    valid_loss_pred.append(accu_pred / valid_set_len)
    valid_loss_comp_pred.append(accu_comp_pred / valid_set_len)

         # Save the best models
        if val_loss_comp[-1] == min(val_loss_comp):
            torch.save(model.state_dict(), "save_model_comp.pt")
            print("Model 'comp' saved for epoch", epoch)
        if val_loss_pred[-1] == min(val_loss_pred):
            torch.save(model.state_dict(), "save_model_pred.pt")
            print("Model 'pred' saved for epoch", epoch)
        if val_loss_comp_pred[-1] == min(val_loss_comp_pred):
            torch.save(model.state_dict(), "save_model_comp_pred.pt")
            print("Model 'comp_pred' saved for epoch", epoch)

    print("Train loss 'comp'", train_loss_comp[-1])
    print("Val loss 'comp'", val_loss_comp[-1])

    print("Train loss 'pred'", train_loss_pred[-1])
    print("Val loss 'pred'", val_loss_pred[-1])

    print("Train loss 'comp_pred'", train_loss_comp_pred[-1])
    print("Val loss 'comp_pred'", val_loss_comp_pred[-1])


In [ ]:
plt.plot(train_loss_comp, label="Train Loss 'comp'")
plt.plot(val_loss_comp, label="Validation Loss 'comp'")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

plt.plot(train_loss_pred, label="Train Loss 'pred'")
plt.plot(val_loss_pred, label="Validation Loss 'pred'")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

lt.plot(train_loss_comp_pred, label="Train Loss 'comp_pred'")
plt.plot(val_loss_comp_pred, label="Validation Loss 'comp_pred'")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()